<a href="https://colab.research.google.com/github/brunoras/nlpjuridico/blob/main/NER_costumizado_para_textos_Jur%C3%ADdicos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 23.6 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
# Dataset
from datasets import load_dataset

# Dependências do SpaCy
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
from spacy.util import filter_spans

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [ ]:
# Carregando o dataset Lener-BR
datasets = load_dataset("lener_br")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/7828 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1177 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1390 [00:00<?, ? examples/s]

Dataset lener_br downloaded and prepared to /root/.cache/huggingface/datasets/lener_br/lener_br/1.0.0/4a8c97e6813b5c2d85a50faf0a3e6c24ea82f4a9044e6e9e8b24997d27399382. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 7828
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 1177
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 1390
    })
})

In [ ]:
# Tags do dataset
datasets["train"].features[f"ner_tags"]

Sequence(feature=ClassLabel(names=['O', 'B-ORGANIZACAO', 'I-ORGANIZACAO', 'B-PESSOA', 'I-PESSOA', 'B-TEMPO', 'I-TEMPO', 'B-LOCAL', 'I-LOCAL', 'B-LEGISLACAO', 'I-LEGISLACAO', 'B-JURISPRUDENCIA', 'I-JURISPRUDENCIA'], id=None), length=-1, id=None)

In [ ]:
classes = ['O', 'ORGANIZACAO', 'ORGANIZACAO', 'PESSOA', 'PESSOA', 'TEMPO', 'TEMPO', 
           'LOCAL', 'LOCAL', 'LEGISLACAO', 'LEGISLACAO', 'JURISPRUDENCIA', 'JURISPRUDENCIA']
punt = ['.', ',', ':', 'º', '/', '%']

In [ ]:
training_data = {'classes': ['ORGANIZACAO', 'PESSOA', 'TEMPO', 'LOCAL', 'LEGISLACAO', 'JURISPRUDENCIA'],
                 'annotations': []}
def montartreino(dataset):
  for data in dataset:
    text = ''
    tagged = ''
    entities = []
    for i in range(len(data['ner_tags'])):
      if data['ner_tags'][i] > 0:
        classe = classes[data['ner_tags'][i]]
        if data['tokens'][i] in punt:
          tagged += data['tokens'][i]
        else:
          tagged += ' ' + data['tokens'][i]
      else:
        if len(tagged) > 0:
          start = len(text) + 1
          end = len(text) + len(tagged)
          entities.append((start, end, classe))
          if data['tokens'][i] in punt:
            text += tagged + data['tokens'][i]
          else:
            text += tagged + ' ' + data['tokens'][i]
          tagged = ''
        else:
          if len(text) > 0:
            if data['tokens'][i] in punt:
              text += data['tokens'][i]
            else:
              text += ' ' + data['tokens'][i]
          else:
            text = data['tokens'][i]
    
    training_data['annotations'].append({'entities': entities, 'text': text})
    
montartreino(datasets['train'])
montartreino(datasets['validation'])
montartreino(datasets['test'])

print(f"{len(training_data['annotations'])} entradas formatadas.")

10395 entradas formatadas.


In [ ]:
training_data['annotations'][1]

{'entities': [(4, 24, 'LEGISLACAO'),
  (50, 68, 'ORGANIZACAO'),
  (195, 213, 'ORGANIZACAO')],
 'text': '- O art. 178, II, do CPC prescreve que compete ao Ministério Público intervir nas causas em que há interesses de incapazes, dispondo o art. 279 do mesmo diploma que o processo será nulo quando o Ministério Público não for intimado para acompanhar o feito em que deve intervir.'}

In [ ]:
# Criando um objeto spaCy vazio
nlp = spacy.blank('pt')

# Criando um objeto DocBin
doc_bin = DocBin()

In [ ]:
# Removendo entradas mau formatadas, pois o objeto para treino precisa de formatação específica
for training_example  in tqdm(training_data['annotations']): 
  text = training_example['text']
  labels = training_example['entities']
  doc = nlp.make_doc(text) 
  ents = []
  for start, end, label in labels:
    span = doc.char_span(start, end, label=label, alignment_mode="contract")
    if span is None:
      print("Skipping entity")
    else:
      ents.append(span)
  filtered_ents = filter_spans(ents)
  doc.ents = filtered_ents 
  doc_bin.add(doc)
doc_bin.to_disk("training_data.spacy") # salvando o objeto docbin

 88%|████████▊ | 9190/10395 [00:03<00:00, 2491.18it/s]

Skipping entity


100%|██████████| 10395/10395 [00:04<00:00, 2454.13it/s]


Carregando o arquivo de configuração gerado em https://spacy.io/usage/training#quickstart

As configurações são:

Language: Português

Components: ner

Hardware: CPU

Optimize for: efficiency

In [ ]:
!python -m spacy init fill-config base_config.cfg config.cfg

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2022-12-27 12:04:23.940968: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


Treinando:

In [ ]:
!python -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./training_data.spacy

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2022-12-27 12:04:33.605359: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2022-12-27 12:04:34,340] [INFO] Set up nlp object from config
INFO:spacy:Set up nlp object from config
[2022-12-27 12:04:34,352] [INFO] Pipeline: ['tok2vec', 'ner']
INFO:spacy:Pipeline: ['tok2vec', 'ner']
[2022-12-27 12:04:34,357] [INFO] Created vocabulary
INFO:spacy:Created vocabulary
[2022-12-27 12:04:34,358] [INFO] Finished initializing nlp object
INFO:spacy:Finished initializing nlp object
[2022-12-27 12:04:41,949] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
INFO:spacy:Initialized pipeline components: ['tok2vec', 'ner']
✔ Initialized pipeline

==

Teste com uma jurisprudência retirada do TJ-BA

In [ ]:
meucaso = "Tribunal de Justiça do Estado da Bahia PODER JUDICIÁRIO QUARTA TURMA RECURSAL - PROJUDI PADRE CASIMIRO QUIROGA, LT. RIO DAS PEDRAS, QD 01, SALVADOR - BA ssa-turmasrecursais@tjba.jus.br - Tel.: 71 3372-7460 Ação: Procedimento do Juizado Especial Cível Recurso nº 0009125-62.2022.8.05.0080 Processo nº 0009125-62.2022.8.05.0080 Recorrente(s): DOS SANTOS NASCIMENTO Recorrido(s): 123 MILHAS VIAGENS E TURISMO LTDA GOL LINHAS AEREAS S A DECISÃO MONOCRÁTICA RECURSO INOMINADO. O NOVO REGIMENTO DAS TURMAS RECURSAIS, RESOLUÇÃO Nº 02/2021, ESTABELECEU A COMPETÊNCIA DO RELATOR PARA JULGAR MONOCRATICAMENTE MATÉRIAS COM UNIFORMIZAÇÃO DE JURISPRUDÊNCIA OU ENTENDIMENTO SEDIMENTADO. CONDIÇÕES DE ADMISSIBILIDADE PREENCHIDAS. AÇÃO INDENIZATÓRIA EM RAZÃO DE SUPOSTA FALHA NA PRESTAÇÃO DE SERVIÇO DE TRANSPORTE AÉREO. AUSÊNCIA DE PROVA ADEQUADA DOS FATOS ALEGADOS. SENTENÇA DE IMPROCEDÊNCIA MANTIDA. RECURSO DO AUTOR CONHECIDO E NÃO PROVIDO. A Resolução nº 02, de 10 de fevereiro de 2021, que instituiu o Regimento Interno das Turmas Recursais dos Juizados Especiais Cíveis, Criminais e da Fazenda Pública do Estado da Bahia e da Turma de Uniformização da Jurisprudência, estabeleceu a competência do relator para julgar monocraticamente matérias com uniformização de jurisprudência ou entendimento sedimentado. Trata-se de ação indenizatória decorrente de vício do serviço. Tentativa de obtenção reembolso de passagem aérea. A ré contestou o feito. Nega conduta indevida e dever de indenizar. Sentença proferida nos seguintes termos: “Assim, ante o exposto e tudo mais que dos autos consta, JULGO IMPROCEDENTE a queixa prestada por JEANE DOS SANTOS NASCIMENTO contra GOL LINHAS AEREAS S/A e 123 VIAGENS E TURISMO LTDA. Sem custas e sem honorários nessa fase processual, por força da Lei nº. 9.099/95.” Irresignado, o autor recorrente busca a procedência total de sua demanda. Entende o STJ que, em se tratando de produção de provas, a inversão, em caso de relação de consumo, não é automática, cabendo ao magistrado a apreciação dos aspectos de verossimilhança da alegação do consumidor ou de sua hipossuficiência, conforme estabelece o art. 6º, VIII, do referido diploma legal. Pois bem, a teor do art. 373, I, do NCPC, é da parte autora o ônus da prova quanto ao fato constitutivo do seu direito. E, mesmo que se considere como sendo de consumo a relação em discussão, a ser amparada pelo Código de Defesa do Consumidor, a inversão do ônus da prova em benefício do consumidor somente deve ser determinada, quando, a critério do juiz, for verossímil a alegação ou quando for ele - o consumidor - hipossuficiente (art. 6º, VIII do CDC). No caso em tela, entendo que a parte autora não logrou êxito em demonstrar os fatos constitutivos de seu direito. O elemento central da tese autoral esbarra na insuficiência probatória, não tendo feito comprovação de atendimento aos requisitos para obtenção de reembolso, nos termos do CC: Art. 740. O passageiro tem direito a rescindir o contrato de transporte antes de iniciada a viagem, sendo-lhe devida a restituição do valor da passagem, desde que feita a comunicação ao transportador em tempo de ser renegociada. Logo, ausente tal comprovação, a pretensão autoral resta prejudicada. Após a leitura das razões recursais, este juízo conclui que a sentença merece manutenção, uma vez que não há argumentos individualizados em evidências, capazes de afastar as conclusões do juízo de origem. A esse respeito, caberia ao recorrente a comprovação de atendimento aos standards probatórios, de modo individualizado, em suas razões. Standards de prova “são critérios que estabelecem o grau de confirmação probatória necessário para que o julgador considere um enunciado fático como provado” (BADARÓ, Gustavo H. Epistemologia judiciária e prova penal. São Paulo: RT, 2019, p. 236). No mesmo sentido, as razões da origem: “Entretanto, cabe à parte autora a prova da solicitação de cancelamento prévio à data do embarque, inclusive para permitir às empresas a comercialização do assento cujo passageiro desistiu de viajar.   Observando os documentos trazidos pela parte autora, não encontrei o pedido de cancelamento alegado. Não há como ter certeza de se e quando foi realizada a solicitação mencionada.   Ora, é ônus da parte autora a prova do fato constitutivo do seu direito, que, neste caso, é a solicitação prévia de cancelamento.”. Nesse sentido: AGRAVO INTERNO NO RECURSO ESPECIAL. AÇÃO DECLARATÓRIA DE INEXISTÊNCIA DE DÉBITO C/C REPARAÇÃO POR DANOS MORAIS. ALEGAÇÃO DE OMISSÃO E DEFICIÊNCIA DE FUNDAMENTAÇÃO. NÃO OCORRÊNCIA. INVERSÃO DO ÔNUS DA PROVA. NECESSIDADE DE COMPROVAÇÃO MÍNIMA DAS TESES DEDUZIDAS. AGRAVO DESPROVIDO. 1. O decisum recorrido esclareceu adequadamente a controvérsia, apontando justificação consistente, não se confundindo com omissão ou deficiência de fundamentação o simples fato de ter apresentado embasamento diferente do pretendido pela parte. 2. A jurisprudência desta Corte Superior se posiciona no sentido de que a inversão do ônus da prova não dispensa a comprovação mínima, pela parte autora, dos fatos constitutivos do seu direito. 3. Assim, antes de ser imputado à ré o ônus de produção da prova em sentido contrário, caberia ao autor comprovar minimamente o seu direito, por meio da apresentação de documento comprobatório do pedido de cancelamento do terminal telefônico, ônus do qual não desincumbiu. 4. Agravo interno desprovido. (SRJ, AgInt no REsp 1717781/RO, Rel. Ministro MARCO AURÉLIO BELLIZZE, TERCEIRA TURMA, julgado em 05/06/2018, DJe 15/06/2018) [grifos nossos] APELAÇÃO CÍVEL. AÇÃO COBRANÇA. ÔNUS DA PROVA. FATO CONSTITUTIVO. AUSÊNCIA. IMPROCEDÊNCIA DA PRETENSÃO. Se a parte autora não se desincumbiu do ônus da prova e não demonstrou o fato constitutivo do seu direito, é caso de se manter a decisão de primeira instância que julgou improcedente o pedido inicial. Aplicação do artigo 373, inciso I, do Código de Processo Civil. Hipótese em que a requerente não demonstrou que os serviços foram efetivamente prestados. (TJMG - Apelação Cível 1.0529.16.003356-7/001, Relator(a): Des.(a) Tiago Pinto , 15ª CÂMARA CÍVEL, julgamento em 22/08/2019, publicação da súmula em 28/08/2019) (grifou-se) Segundo entendimento do Superior Tribunal de Justiça ¿os documentos indispensáveis à propositura da ação, e que devem ser instruídos com a inicial, são aqueles que comprovam a ocorrência da causa de pedir (documentos fundamentais) e, em casos específicos, os que a própria lei exige como da substância do ato que está sendo levado à apreciação (documentos substanciais)¿. (REsp 1040715/DF, Rel. Ministro MASSAMI UYEDA, TERCEIRA TURMA, julgado em 04/05/2010, DJe 20/05/2010).¿ Por essas razões, NEGO PROVIMENTO ao recurso, mantendo a sentença de origem em todos os seus termos, pelos próprios fundamentos. Decisão integrativa nos termos do art. 46 da lei 9.099/95. Custas e honorários pela parte autora/recorrente em 20% sobre o valor da causa, com exigibilidade suspensa em razão do deferimento da assistência judiciária gratuita. Salvador-BA, em 07 de dezembro de 2022.   ELIENE SIMONE SILVA OLIVEIRA Juíza Relatora em Substituição"

In [ ]:
nlp_ner = spacy.load("model-best")

doc = nlp_ner(meucaso)


colors = {'ORGANIZACAO': '#00FFFF', 'PESSOA': '#0000FF', 'TEMPO': '#0BDA51', 
          'LOCAL': '#FF00FF', 'LEGISLACAO': '#7F00FF', 'JURISPRUDENCIA': '#E4D00A'}
options = {"colors": colors}

spacy.displacy.render(doc, style="ent", options=options, jupyter=True)

In [ ]:
!zip -r /content/nermodel.zip /content/model-best

  adding: content/model-best/ (stored 0%)
  adding: content/model-best/vocab/ (stored 0%)
  adding: content/model-best/vocab/strings.json (deflated 76%)
  adding: content/model-best/vocab/key2row (stored 0%)
  adding: content/model-best/vocab/vectors (deflated 45%)
  adding: content/model-best/vocab/vectors.cfg (stored 0%)
  adding: content/model-best/vocab/lookups.bin (stored 0%)
  adding: content/model-best/tok2vec/ (stored 0%)
  adding: content/model-best/tok2vec/cfg (stored 0%)
  adding: content/model-best/tok2vec/model (deflated 7%)
  adding: content/model-best/tokenizer (deflated 84%)
  adding: content/model-best/meta.json (deflated 61%)
  adding: content/model-best/ner/ (stored 0%)
  adding: content/model-best/ner/cfg (deflated 33%)
  adding: content/model-best/ner/model (deflated 7%)
  adding: content/model-best/ner/moves (deflated 68%)
  adding: content/model-best/config.cfg (deflated 61%)
